In [340]:
import importlib
import utils
import transformers
import models
import losses

In [341]:
from models import *
from utils import *

In [342]:
importlib.reload(utils)
importlib.reload(transformers)
importlib.reload(models)
importlib.reload(losses)

<module 'losses' from 'd:\\Documents\\machinelearning\\cgm-paper\\src\\losses.py'>

In [343]:
print("Current working directory: ", os.getcwd())

Current working directory:  d:\Documents\machinelearning\cgm-paper\src


In [344]:
raw_dir = "../data/raw"
tuning_dir = "../data/tuning"

In [345]:
assert os.path.exists(raw_dir), "Raw data directory not found"
assert os.path.exists(tuning_dir), "Tuning data directory not found"

In [346]:
dataset = load_data(raw_dir)

Found 22 files in ../data/raw


train (139415, 47)
test (32912, 47)


In [347]:
val_dataset = load_data(tuning_dir)

Found 2 files in ../data/tuning
train (13630, 43)
test (3002, 43)


In [348]:
for key in dataset.keys():
    val_dataset[key] = reduce_classes(val_dataset[key])
    dataset[key] = reduce_classes(dataset[key])

In [349]:
dataset.get("train")['Time'].isnull().sum()

0

In [350]:
train_data = get_train_dataset(dataset['train'])
test_data = get_any_dataset(dataset['test'], dataset['train'])

      CGM  month  day  hour  minute  weekday  cgm_velo  CGM(1)  CGM(2)  \
6    71.0      5   19    12       5        2  0.010000    66.0    64.5   
7    78.0      5   19    12      10        2  0.016667    71.0    68.5   
8    90.0      5   19    12      15        2  0.023333    78.0    74.5   
9    99.0      5   19    12      20        2  0.040000    90.0    84.0   
10  110.0      5   19    12      25        2  0.030000    99.0    94.5   

       CGM(3)  CGM(4)  CGM(5)     CGM(6)  
6   64.666667   65.50    66.8  68.333333  
7   66.666667   66.25    66.6  67.500000  
8   71.666667   69.50    68.6  68.500000  
9   79.666667   76.25    73.6  72.166667  
10  89.000000   84.50    80.8  77.833333  


The tuning data will be merged with the training data and be fitted into the pipeline to be used for the validation data

In [351]:
tuning_data = get_tuning_dataset(val_dataset['train'], dataset['train'])
val_data = get_any_dataset(val_dataset['test'], tuning_data)

In [352]:
models = Models()

In [353]:
trainX, trainY = train_data.drop(["CGM"], axis = 1), train_data["CGM"]
trainX.shape, trainY.shape

((123697, 12), (123697,))

In [354]:
testX, testY = test_data.drop(["CGM"], axis = 1), test_data["CGM"]
testX.shape, testY.shape

((28455, 12), (28455,))

In [355]:
models.fit(trainX, trainY, testX, testY)

 50%|█████     | 1/2 [00:00<00:00,  3.89it/s]

XGBoost MSE: 21.700712394573824, RMSE: 4.6584023435695014, MSPE: 0.08981187579422874 %


100%|██████████| 2/2 [00:00<00:00,  3.65it/s]

LightGBM MSE: 18.80539329014221, RMSE: 4.3365185679462055, MSPE: 0.08458871243019242 %


In [356]:
N = 12

In [357]:
forecastX, forecastY = testX.iloc[:10], testY.iloc[10: 10 + N]

In [358]:
forecastXLong = testX.iloc[10: 10 + N]

In [364]:
forecastTX, forecastTY = models.forecast(forecastX, forecastY, N)
forecastTX

,month,day,hour,minute,weekday,cgm_velo,CGM(1),CGM(2),CGM(3),CGM(4),CGM(5),CGM(6)
22,7,4,2,20,1,0.001727,212.610229,212.374672,212.282145,212.176269,212.099353,212.094177
23,7,4,2,25,1,0.002437,213.341393,212.858032,212.635228,212.467550,212.347761,212.302046
24,7,4,2,30,1,0.004513,214.695151,213.776592,213.321869,213.024450,212.817239,212.700897
25,7,4,2,35,1,0.006204,216.556484,215.166538,214.400074,213.907459,213.565088,213.343495
26,7,4,2,40,1,0.005563,218.225475,216.696006,215.675208,214.986963,214.497165,214.157158
27,7,4,2,45,1,0.007883,220.590279,218.643143,217.313565,216.387792,215.715788,215.229345
28,7,4,2,50,1,0.005936,222.371094,220.507119,218.999408,217.883618,217.046849,216.419637
29,7,4,2,55,1,0.005842,224.123697,222.315408,220.707504,219.443637,218.462219,217.703647
30,7,4,3,0,1,0.006036,225.934591,224.124999,222.449867,221.066376,219.956693,219.075471
31,7,4,3,5,1,0.002331,226.633972,225.379486,223.844569,222.458275,221.292149,220.335221


In [365]:
forecastTY

,CGM
0,210.097565
1,211.466522
2,212.100418
3,212.189224
4,212.221878
5,212.251419
6,212.282623
7,212.282623
8,212.282623
9,212.154251


In [361]:
forecastXLong.reset_index(drop = True, inplace = True)
forecastXLong

,month,day,hour,minute,weekday,cgm_velo,CGM(1),CGM(2),CGM(3),CGM(4),CGM(5),CGM(6)
0,7,4,1,20,6,0.006667,210.0,209.0,207.666667,208.00,208.8,209.833333
1,7,4,1,25,6,-0.013333,206.0,208.0,208.000000,207.25,207.6,208.333333
2,7,4,1,30,6,-0.016667,201.0,203.5,205.666667,206.25,206.0,206.500000
3,7,4,1,35,6,-0.010000,198.0,199.5,201.666667,203.75,204.6,204.666667
4,7,4,1,40,6,-0.010000,195.0,196.5,198.000000,200.00,202.0,203.000000
5,7,4,1,45,6,-0.010000,192.0,193.5,195.000000,196.50,198.4,200.333333
6,7,4,1,50,6,-0.003333,191.0,191.5,192.666667,194.00,195.4,197.166667
7,7,4,1,55,6,-0.010000,188.0,189.5,190.333333,191.50,192.8,194.166667
8,7,4,2,0,6,-0.010000,185.0,186.5,188.000000,189.00,190.2,191.500000
9,7,4,2,5,6,-0.003333,184.0,184.5,185.666667,187.00,188.0,189.166667
